### Exercício 2 - Projeto 3 - Lógica Computacional

Grupo G20

Nome: Lara Catarina Vilaça Lopes ; Número de aluno:a108655

Nome: Fábio Mendes Castelhano ; Número de aluno:a105728

1. Algoritmo de Euclides

INPUT  a, b

assume  a > 0 and b > 0

r, r', s, s', t, t' = a, b, 1, 0, 0, 1

while r' != 0
    q = r div r'

    r, r', s, s', t, t' = r', r - q × r', s', s - q × s', t', t - q × t'

OUTPUT r, s, t


As variáveis auxiliares `r'`, `s'`  e `t'` foram representadas no programa pelos identificadores `rl`, `sl` e `tl`, respetivamente. 


Para provar a correção do Algoritmo de Euclides vamos recorrer ao uso da biblioteca pysmt

In [9]:
from pysmt.shortcuts import *
from pysmt.typing import INT

2. Construção do CFA

Na função `declare` estamos a **criar variáveis simbólicas** que representam o estado do programa no passo i.

In [10]:
def declare(i):
    state = {}
    state['pc'] = Symbol('pc'+str(i), INT)
    state['r'] = Symbol('r'+ str(i), INT)
    state['rl'] = Symbol('rl' + str(i), INT)
    state['s'] = Symbol('s'+ str(i), INT)
    state['sl'] = Symbol('sl' + str(i), INT)
    state['t'] = Symbol('t' + str(i), INT)
    state['tl'] = Symbol('tl' + str(i), INT)
    state['a'] = Symbol('a' + str(i), INT)
    state['b'] = Symbol('b' + str(i), INT)
    return state

Na função `init` estamos a definir o **estado inicial** do programa, ou seja, os valores das variáveis simbólicas antes da execução do algoritmo:
    
- **pc = 0**
- **a > 0**
- **b > 0**
- **r = a**
- **rl = b**
- **s = 1**  
- **sl = 0**  
- **t = 0**  
- **tl = 1**  


In [11]:
def init(state):
    return And(
        Equals(state['pc'], Int(0)),
        GT(state['a'] ,Int(0)),
        GT(state['b'], Int(0)),
        Equals(state['r'], state['a']),
        Equals(state['rl'], state['b']),
        Equals(state['s'], Int(1)),
        Equals(state['sl'], Int(0)),
        Equals(state['t'], Int(0)),
        Equals(state['tl'], Int(1))
        )


Na função `transitions` definimos os diferentes estados do programa e como se passam de um para outro:

- **l_start → l_cond**  
  Transição do estado inicial (`pc == 0`) para a condição do ciclo while(`pc == 1`).

- **l_cond → l_cycle**  
  Quando a condição do ciclo while (`rl != 0`) é verdadeira, passa-se para o estado do ciclo while(`pc == 2`).

- **l_cycle → l_cond**  
  Depois de executar o ciclo while, voltamos à condição do ciclo while para a próxima iteração ( `pc == 1`).

- **l_cond → l_end**  
  Quando a condição do ciclo while (`rl == 0`) é falsa, o programa termina ( `pc == 3`).


`Guardas` definidas nas transições:

- **Transição `l_cond → l_cycle`**  
  - De `pc == 1` para `pc == 2`  
  - **Guarda:** `rl != 0`

- **Transição `l_cond → l_end`**  
  - De `pc == 1` para `pc == 3`  
  - **Guarda:** `rl == 0`


Local de `terminação` com sucesso:
- **`l_end (pc = 3)`**  
  Representa a terminação correta do programa quando `rl == 0`.


In [12]:
def transitions(curr,next):
        return Or(
                #l_start -> l_cond
                And(
                    Equals(curr['pc'], Int(0)),
                    Equals(next['pc'],Int(1)),
                    Equals(next['r'], curr['r']),
                    Equals(next['rl'], curr['rl']),
                    Equals(next['s'],curr['s']),
                    Equals(next['sl'], curr['sl']),
                    Equals(next['t'],curr['t']),
                    Equals(next['tl'],curr['tl']),
                    Equals(next['a'],curr['a']),
                    Equals(next['b'],curr['b'])
                    ),
                #l_cond -> l_cycle
                And(
                    Equals(curr['pc'],Int(1)),
                    Not(Equals(curr['rl'],Int(0))), # guarda
                    Equals(next['pc'], Int(2)),
                    Equals(next['r'], curr['r']),
                    Equals(next['rl'], curr['rl']),
                    Equals(next['s'],curr['s']),
                    Equals(next['sl'], curr['sl']),
                    Equals(next['t'],curr['t']),
                    Equals(next['tl'],curr['tl']),
                    Equals(next['a'],curr['a']),
                    Equals(next['b'],curr['b'])
                ),
                #l_cycle -> l_cond
                And(
                    Equals(curr['pc'],Int(2)),
                    Equals(next['pc'],Int(1)),
                    Equals(next['r'],curr['rl']),
                    Equals(next['rl'], Minus(curr['r'], Times(Div(curr['r'], curr['rl']), curr['rl']))),
                    Equals(next['s'], curr['sl']),
                    Equals(next['sl'], Minus(curr['s'], Times(Div(curr['r'], curr['rl']),curr['sl']))),
                    Equals(next['t'], curr['tl']),
                    Equals(next['tl'],Minus ( curr['t'], Times(Div(curr['r'], curr['rl']) ,curr['tl']))),
                    Equals(next['a'],curr['a']),
                    Equals(next['b'],curr['b'])
                ),
                #l_cond -> l_end
                And(
                    Equals(curr['pc'],Int(1)),
                    Equals(curr['rl'],Int(0)), #guarda
                    Equals(next['pc'],Int(3)),
                    Equals(next['r'], curr['r']),
                    Equals(next['rl'], curr['rl']),
                    Equals(next['s'],curr['s']),
                    Equals(next['sl'], curr['sl']),
                    Equals(next['t'],curr['t']),
                    Equals(next['tl'],curr['tl']),
                    Equals(next['a'],curr['a']),
                    Equals(next['b'],curr['b'])
                )
        )

3. Provar que  $$\,\phi(a,b,r,s,t) \,\equiv\; a*s + b*t = r\;$$ é invariante

1. **Combinação linear principal**
   - Fórmula: `a*s + b*t = r`
   - As variáveis `s` e `t`, ponderadas por `a` e `b`, mantêm a igualdade com `r`.

2. **Combinação linear secundária**
   - Fórmula: `a*sl + b*tl = rl`
   - As variáveis auxiliares `sl` e `tl`, ponderadas por `a` e `b`, mantêm a igualdade com `rl`.

3. **Não negatividade**
   - Fórmulas: `r >= 0` e `rl >= 0`
   - As variáveis `r` e `rl` nunca assumem valores negativos durante a execução.

In [13]:
def invariant(s):
    return And(
        # a*s + b*t = r
        Equals(
            Plus(Times(s['a'], s['s']), Times(s['b'], s['t'])),
            s['r']
        ),
        # a*sl + b*tl = rl
        Equals(
            Plus(Times(s['a'], s['sl']), Times(s['b'], s['tl'])),
            s['rl']
        ),

        # r >= 0  e  rl >= 0
        GE(s['r'], Int(0)),
        GE(s['rl'], Int(0)),

    )


A função `k_induction` implementa a metodologia de **k-indução** para verificar se um invariante se mantém em todas as execuções do programa:
- Divide a prova em duas partes: **base case** (validade inicial) e **inductive step** (preservação).
- Se ambos forem satisfeitos, o invariante está formalmente verificado.


In [14]:
def k_induction(declare,init,transitions,k):
    with Solver(name = "z3") as solver:
        states = [declare(i) for i in range(k)]

        base = [init(states[0])]

        for i in range(k-1):
            base.append(transitions(states[i],states[i+1]))

        base_case = And(And(base), Or([Not(invariant(s)) for s in states]))

        solver.add_assertion(base_case)
        if solver.solve():
            print("The base case does not respect the invariant.")
            print("CE: ", solver.get_model())
            return False
        solver.reset_assertions()

        states = [declare(i) for i in range(k+1)]
        inductive = []
        for i in range(k):
            inductive.append(transitions(states[i], states[i+1]))
        inductive.append(And([invariant(states[i]) for i in range(k)]))
        inductive.append(Not(invariant(states[k])))

        step = And(inductive)
        solver.add_assertion(step)
        if solver.solve():
            print("The inductive step does not respect the invariant.")
            print("CE: ", solver.get_model())
            return False
    print("Invariant verified by k-induction.")
    return True

k_induction(declare, init, transitions, 2)

Invariant verified by k-induction.


True

4. Usando a metodologia dos “look-aheads” vamos provar que o programa termina sempre

Na função `variant`, o **variante** escolhido é a variável `rl`.

- O variante é usado para demonstrar **terminação** do ciclo.
- Enquanto `rl != 0`, o ciclo continua (`l_cond → l_cycle`).
- Em cada passo do ciclo (`l_cycle → l_cond`), `rl` é atualizado para um valor **estritamente menor** ou **igual**.
- Eventualmente, `rl` atinge o valor `0`.
- Quando `rl = 0`, a guarda leva o programa para o estado final `l_end`.

In [15]:
def variant(state):
    return state['rl']

Como já encontramos um variante simples podemos verificar a terminação do programa usando a metadologia "look-aheads":

- O variante `rl` só é garantidamente atualizado de forma decrescente após a sequência completa:
  `l_cond → l_cycle → l_cond`.
- Em apenas 2 passos, `k = 2`, o solver encontra estados onde `rl` ainda não diminuiu.
- Para `k = 3`, já tem a sequência completa e o decréscimo de `rl` é assegurado.
- Assim, a prova de terminação precisa de `k >= 3` para ser válida.
- Isto garante que o ciclo não é infinito e que o programa termina corretamente quando `rl = 0`.

In [16]:
def k_lookahead(declare, init, transitions, var, k):
    with Solver(name="z3") as s:
        states = [declare(i) for i in range(k+1)]

        s.add_assertion(init(states[0]))

        transitions_list = [transitions(states[i], states[i+1]) for i in range(k)]

        all_transitions = And(transitions_list)

        decreases = Or(
                       LT(var(states[-1]), var(states[0])),
                       Equals(var(states[-1]), Int(0))    
                    )
        
        s.add_assertion(And(all_transitions, Not(decreases)))

        if s.solve():
            print(f"There is a sequence of {k} steps where the variant does not decrease.")
            print("CE: ",s.get_model())
            return False
        
        print(f"The variant decreases every {k} steps, or reaches 0.")


k_lookahead(declare,init,transitions,variant,3)

The variant decreases every 3 steps, or reaches 0.


In [17]:
def k_lookahead(declare, init, transitions, var, k):
    with Solver(name="z3") as s:
        states = [declare(i) for i in range(k+1)]

        s.add_assertion(init(states[0]))

        transitions_list = [transitions(states[i], states[i+1]) for i in range(k)]

        all_transitions = And(transitions_list)

        decreases = Or(
                       LT(var(states[-1]), var(states[0])),
                       Equals(var(states[-1]), Int(0))    
                    )
        
        s.add_assertion(And(all_transitions, Not(decreases)))

        if s.solve():
            print(f"There is a sequence of {k} steps where the variant does not decrease.")
            print("CE: ",s.get_model())
            return False
        
        print(f"The variant decreases every {k} steps, or reaches 0.")


k_lookahead(declare,init,transitions,variant,2)

There is a sequence of 2 steps where the variant does not decrease.
CE:  t2 := 0
sl1 := 0
pc0 := 0
tl2 := 1
t1 := 0
s0 := 1
a0 := 1
a2 := 1
tl1 := 1
sl0 := 0
b2 := 2
pc2 := 2
t0 := 0
r1 := 1
r2 := 1
tl0 := 1
a1 := 1
s2 := 1
pc1 := 1
b1 := 2
sl2 := 0
s1 := 1
r0 := 1
rl0 := 2
rl1 := 2
rl2 := 2
b0 := 2


False